Assignment 2 - Deep Learning

By: Leo Tsang

In [2]:
#connect to our Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#import all the packages required to read image
import numpy as np
np.random.seed(1234)
import skimage
from skimage import io
from skimage.color import rgb2gray, rgb2grey
from skimage.transform import resize
from skimage.io import imread_collection, concatenate_images, imshow
from skimage.util import invert

import matplotlib.pyplot as plt

In [0]:
#listing the directory 
train_dir = '/content/drive/My Drive/DS8009 Deep Learning/Assignment 2/training/'
valid_dir = '/content/drive/My Drive/DS8009 Deep Learning/Assignment 2/evaluation/'
test_dir = '/content/drive/My Drive/DS8009 Deep Learning/Assignment 2/validation/'


PRETRAIN Neural Network

In [5]:
#let's try pretrain
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 2s 0us/step


In [6]:
#Pre-processing & Setting images to batches

import numpy as np
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 3000)
validation_features, validation_labels = extract_features(valid_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 3000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
#setting the image to fit our model

train_features = np.reshape(train_features, (3000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [8]:
from keras import models
from keras import layers
from keras import optimizers

model2 = models.Sequential()
model2.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history2 = model2.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3000 samples, validate on 1000 samples
Epoch 1/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.4729 - acc: 0.7707 - val_loss: 0.2732 - val_acc: 0.9030
Epoch 2/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.2600 - acc: 0.9040 - val_loss: 0.2211 - val_acc: 0.9170
Epoch 3/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.2119 - acc: 0.9183 - val_loss: 0.1994 - val_acc: 0.9260
Epoch 4/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.1916 - acc: 0.9250 - val_loss: 0.1864 - val_acc: 0.9310
Epoch 5/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.1670 - acc: 0.9360 - val_loss: 0.1787 - val_acc: 0.9340
Epoch 6/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.1569 - acc: 0.9393 - val_loss: 0.1737 - v

In [9]:
test = model2.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(test_features, test_labels))

Train on 3000 samples, validate on 1000 samples
Epoch 1/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0381 - acc: 0.9900 - val_loss: 0.1308 - val_acc: 0.9420
Epoch 2/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0371 - acc: 0.9903 - val_loss: 0.1321 - val_acc: 0.9440
Epoch 3/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0355 - acc: 0.9913 - val_loss: 0.1401 - val_acc: 0.9470
Epoch 4/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0339 - acc: 0.9923 - val_loss: 0.1323 - val_acc: 0.9430
Epoch 5/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0311 - acc: 0.9923 - val_loss: 0.1441 - val_acc: 0.9490
Epoch 6/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0288 - acc: 0.9950 - val_loss: 0.1332 - val_acc: 0.9410
Epoch 7/30
3000/3000 [==============================] - 8s 3ms/step - loss: 0.0278 - acc: 0.9940 - val_loss: 0.1394 - val_acc: 0.9450
Epoch 8/30
300

In [11]:
#our test accuracy on VGG16
test_loss, test_acc = model2.evaluate(test_features, test_labels)
print('test_acc:', test_acc)

1000/1000 [==============================] - 0s 263us/step
test_acc: 0.94


Our pretrain model with VGG16 yields 94%!!

let's try Pretrain with VGG19 and see whether it's better

In [12]:
#let's try pretrain VGG19
from keras.applications import VGG19

conv_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

80142336/80134624 [==============================] - 1s 0us/step


In [13]:
#pretrain model 3
from keras import models
from keras import layers
from keras import optimizers

model3 = models.Sequential()
model3.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model3.add(layers.Dense(512, activation='relu'))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(1, activation='sigmoid'))

model3.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history3 = model2.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

Train on 3000 samples, validate on 1000 samples
Epoch 1/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0075 - acc: 0.9990 - val_loss: 0.1971 - val_acc: 0.9430
Epoch 2/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0081 - acc: 0.9987 - val_loss: 0.1943 - val_acc: 0.9460
Epoch 3/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0072 - acc: 0.9983 - val_loss: 0.1992 - val_acc: 0.9470
Epoch 4/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0074 - acc: 0.9983 - val_loss: 0.2013 - val_acc: 0.9450
Epoch 5/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0062 - acc: 0.9990 - val_loss: 0.2016 - val_acc: 0.9470
Epoch 6/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0068 - acc: 0.9990 - val_loss: 0.2017 - val_acc: 0.9460
Epoch 7/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.0051 - acc: 0.9990 - val_loss: 0.2037 - val_acc: 0.9450
Epoch 8/30
300

In [14]:
test2 = model3.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(test_features, test_labels))

Train on 3000 samples, validate on 1000 samples
Epoch 1/30
3000/3000 [==============================] - 7s 2ms/step - loss: 0.4480 - acc: 0.8163 - val_loss: 0.2645 - val_acc: 0.9150
Epoch 2/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.2597 - acc: 0.9050 - val_loss: 0.1926 - val_acc: 0.9280
Epoch 3/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.2046 - acc: 0.9270 - val_loss: 0.1681 - val_acc: 0.9310
Epoch 4/30
3000/3000 [==============================] - 7s 2ms/step - loss: 0.1772 - acc: 0.9317 - val_loss: 0.1615 - val_acc: 0.9360
Epoch 5/30
3000/3000 [==============================] - 7s 2ms/step - loss: 0.1510 - acc: 0.9460 - val_loss: 0.1582 - val_acc: 0.9360
Epoch 6/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.1335 - acc: 0.9543 - val_loss: 0.1431 - val_acc: 0.9390
Epoch 7/30
3000/3000 [==============================] - 6s 2ms/step - loss: 0.1197 - acc: 0.9567 - val_loss: 0.1387 - val_acc: 0.9410
Epoch 8/30
300

In [15]:
#our test accuracy on VGG19

test_loss2, test_acc2 = model3.evaluate(test_features, test_labels)
print('test_acc:', test_acc2)

1000/1000 [==============================] - 0s 248us/step
test_acc: 0.949


It seems pretrain with VGG19 is a slightly better model at 94.9% accuracy
